In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [9]:
arr = np.genfromtxt("C:/Users/10273/Desktop/data/ctsten0270_input_6/distance_6.txt", delimiter=',')

In [10]:
demand = np.genfromtxt("C:/Users/10273/Desktop/data/ctsten0270_input_6/demand_6.txt",dtype =int, delimiter=',')

In [11]:
demand[:,1]

array([0, 6, 1, ..., 5, 2, 7])

In [12]:
demand = demand[:,1]

In [13]:
demand = demand.transpose()

In [14]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = arr
    data['demands'] = demand
    data['vehicle_capacities'] = [131]*43
    data['num_vehicles'] = 43
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(30)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Objective: 80935
Route for vehicle 0:
 0 Load(0) ->  199 Load(1) ->  67 Load(2) ->  556 Load(11) ->  319 Load(12) ->  763 Load(16) ->  184 Load(17) ->  323 Load(26) ->  618 Load(36) ->  169 Load(40) ->  127 Load(44) ->  137 Load(47) ->  996 Load(52) ->  521 Load(61) ->  612 Load(62) ->  383 Load(64) ->  669 Load(69) ->  749 Load(71) ->  415 Load(72) ->  161 Load(82) ->  286 Load(83) ->  795 Load(87) ->  576 Load(88) ->  404 Load(91) ->  965 Load(93) ->  718 Load(98) ->  603 Load(103) ->  38 Load(112) ->  525 Load(119) ->  962 Load(126) ->  879 Load(131) ->  0 Load(131)
Distance of the route: 3112m
Load of the route: 131

Route for vehicle 1:
 0 Load(0) ->  896 Load(9) ->  275 Load(13) ->  336 Load(23) ->  193 Load(31) ->  166 Load(35) ->  951 Load(40) ->  791 Load(48) ->  276 Load(54) ->  4 Load(61) ->  986 Load(68) ->  281 Load(70) ->  891 Load(74) ->  106 Load(81) ->  496 Load(85) ->  46 Load(87) ->  977 Load(97) ->  233 Load(99) ->  503 Load(104) ->  700 Load(110) ->  504 Load(118) 